In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('config.cfg')
import requests
import time
import pickle
from collections import Counter
from dateutil.parser import parse as dateutil_parse
import dateutil
import pandas as pd
from IPython.display import display
import ujson as json
from datetime import datetime
import json

In [ ]:
key = config['API']['key']

In [ ]:
key

### Article Search Endpoint

In [ ]:
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json" 
params = {'api-key':key
          ,'q':'Underscore'
          ,'begin_date':20160101
          ,'end_date':20161221
          ,'page':0}
r = requests.get(url,params=params)
results = r.json()
# get the number of restuls
total_results = results['response']['meta']['hits']
# create integer for total pages
total_pages = int(round(int(total_results),0))
# prepare the documents
docs = results['response']['docs']
# create a list of lists of documents (page,doc)
res_by_date = [(doc['pub_date'],doc) for doc in docs]
# pack the results list with the page's documents
results_by_date = []
results_by_date.extend(res_by_date)

In [ ]:
for p in range(1,total_pages):
    # update to new page
    params['page']=p
    # get data
    time.sleep(5.5)
    r = requests.get(url,params=params)
    results = r.json()
    # grab the docs
    docs = results['response']['docs']
    # orgaize docs by (date,doc)
    res_by_date = [(doc['pub_date'],doc) for doc in docs]
    # append results
    results_by_date.extend(res_by_date)
    

In [ ]:
len(results_by_date)

In [ ]:
results['response']['meta']['hits']

In [ ]:
with open('./data/2016-12-18_520recs_list_datetup_obj.pkl', 'wb') as fp:
    pickle.dump(results_by_date, fp)

In [ ]:
filepath = './data/2016-12-18_520recs_list_datetup.json'
with open(filepath, 'w') as file_handler:
    for item in results_by_date:
        file_handler.write("{}\n".format(item))

In [ ]:
results['response']['docs'][1]

In [ ]:
sources =[]
for date,doc in results_by_date:
    sources.append(doc['source'])
    

In [ ]:
len(results_by_date)

### What are the publication sources?

In [ ]:
Counter(sources)

### What is the date range?

In [ ]:
dates = set()
for date,record in results_by_date:
    dates.add(dateutil_parse(date))

In [ ]:
print (max(dates))
print (min(dates))

### On what day are articles published?
Build a heatmap similar to [D3 v4 Calendar View](https://bl.ocks.org/micahstubbs/89c6bd879d64aa511372064c6cf85711)
<pre>
Date,Open,High,Low,Close,Volume,Adj Close
2010-10-01,10789.72,10907.41,10759.14,10829.68,4298910000,10829.68
2010-09-30,10835.96,10960.99,10732.27,10788.05,4284160000,10788.05
...
</pre>

In [ ]:
!head -n1 data/2016-12-18_520recs_list_datetup.json

In [ ]:
date_headline_tup = []
for date, rec in results_by_date:
    date_obj = dateutil_parse(date)
    headline = rec['headline']['main']
    date_headline_tup.append({'date':date_obj,'headline':headline})
df=pd.DataFrame(date_headline_tup)
df.index=df.date
df['value'] = 1
print(df.dtypes)
display(df.head())

In [ ]:
weekly_sum= df.resample('W').sum()

In [ ]:
df['weekday_name']=df.date.dt.weekday_name

In [ ]:
daily = df.date.dt.weekday_name
daily.value_counts()

In [ ]:
hourly = df.date.dt.hour
hourly.value_counts()

In [ ]:
len(results_by_date)

In [ ]:
!ls

In [ ]:
!jq data/2016-12-18_520recs_list_datetup.json 

### Normalize: Archived NYTimes

In [ ]:
def nyt_query(url= "https://api.nytimes.com/svc/search/v2/articlesearch.json" 
              , params =
                  {'api-key':key
                  ,'q':'Underscore'
                  ,'begin_date':20160101
                  ,'end_date':20161221
                  ,'page':0}
              , tup_date_doc = False
              , fileName = 'data/{}_archive_results.json'.format(datetime.now().strftime('%Y-%m-%d'))
         ):
    
    def eval():
        if tup_date_doc:
            res_by_date = [{"pub_date": doc['pub_date'],"rec":json.dumps(doc)} for doc in docs]
        else:
            res_by_date = docs
            return res_by_date
    # make initial request
    r = requests.get(url,params=params)
    results = r.json()
    # get the number of restuls
    total_results = results['response']['meta']['hits']
    # create integer for total pages
    results_per_page = 10
    total_pages = int(round(int(total_results)/results_per_page,0))
    # prepare the documents
    docs = results['response']['docs']
    # orgaize docs 
    res_by_date = eval()
    # pack the results list with the page's documents
    results_by_date = []
    results_by_date.extend(res_by_date)
    with open(fileName,'w') as f:
        for res in res_by_date:
            f.write(json.dumps(res)+'\n')
        print (total_pages)
        for p in range(1,total_pages):
            print("current page: {}".format(10*p))
            # update to new page
            params['page']=p
            # get data
            time.sleep(1.5)
            r = requests.get(url,params=params)
            try: 
                results = r.json()
            except ValueError as e:
                print('ValueError: {}'.format(str(e)))
                continue
            # grab the docs
            docs = results['response']['docs']
            # orgaize docs 
            res_by_date = eval()
            # append results
            #results_by_date.extend(res_by_date)
            # write results
            for res in res_by_date:
                f.write(json.dumps(res))
    

In [ ]:
year = str(2016)
month = str(2) #(January ==1)
url = "https://api.nytimes.com/svc/archive/v1/YEAR/MONTH.json".replace('YEAR',year).replace('MONTH',month)
params = ({'api-key':key,'page':0})
nyt_query(url=url,params=params)

In [ ]:
len(results_by_date)

In [ ]:
! wc -l data/2016-12-22_archive_results.json

In [ ]:
res_by_date[0]

In [ ]:
!wc -l data/2016-12-22_archive_results.json

In [ ]:
!wc -l data/2016-12-22_headn7413_archive_results.json

### Normalized counts
This data come from all articles printed during the month of January in 2016.

In [ ]:
new_date_headline_tup = []
with open('data/2016-12-22_headn7413_archive_results.json','r') as f:
    for item in f:
        rec = json.loads(item)
        pub_date = dateutil_parse(rec['pub_date'])
        headline = rec['headline']['main']
        new_date_headline_tup.append({'date':date_obj,'headline':headline})
df2=pd.DataFrame(new_date_headline_tup)
df2.index=df2.date
df2['value'] = 1
df2['weekday_name']=df2.date.dt.weekday_name
display(df2.head())

In [ ]:
daily = df2.date.dt.weekday_name
#daily = df.date.dt.weekday_name
daily.value_counts()

In [ ]:
df2.tail()

In [ ]:
!wc -l data/2016-12-24_archive_results.json

In [ ]:
tup_lst = []
with open('data/2016-12-22_headn7413_archive_results.json','r') as f:
    for item in f:
        rec = json.loads(item)
        pub_date = dateutil_parse(rec['pub_date'])
        headline = rec['headline']['main']
        new_date_headline_tup.append({'date':date_obj,'headline':headline})
df2=pd.DataFrame(new_date_headline_tup)
df2.index=df2.date
df2['value'] = 1
df2['weekday_name']=df2.date.dt.weekday_name
display(df2.head())

### Parsing XML from NYT

In [1]:
from lxml import html
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
url = 'https://www.nytimes.com/interactive/2016/01/28/upshot/donald-trump-twitter-insults.html'
file_name = 'data/2017-01-26_test_page.html'
page = requests.get(url)
#print(page.text)
#headers = {'Accept-Encoding': 'identity'}
#r = requests.get(url, headers=headers)
#print(r)
#tree = html.fromstring(page.content)
data = page.text

with open(file_name,'w') as f:
    f.write(data)
    
soup = BeautifulSoup(data)

#for link in soup.find_all('a'):
#    print(link.get('href'))



In [ ]:
PYTHONPATH='/Users/blehman/workspace/github/2016-12-16_NYTimesAPI'

In [ ]:
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
browser.quit()

In [ ]:
print(href)
href.startswith('https://twitter.com/realDonaldTrump/status/')

In [ ]:
soup = BeautifulSoup(html)
tweets = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href==None:
        continue
    elif href.startswith('https://twitter.com/realDonaldTrump/status/'):
        tweets.append(href)

In [ ]:
html
html

In [ ]:
from IPython.display import HTML
HTML('<iframe src={} width=700 height=500></iframe>'.format(url))

In [ ]:
soup = BeautifulSoup(html,"lxml")
tweets = []
for div in soup.find_all(class_="g-insult-links-c",limit=1):
    print(div)


In [ ]:
soup = BeautifulSoup(html,"lxml")
tweets = []
for a in soup.select('.g-insult-links-c a[href^="https://twitter.com/realDonaldTrump/status/"]'):
    name = a.parent.parent.parent.select('.g-entity-name')[0].string
    title = a.parent.parent.parent.select('.g-entity-title')[0].string
    link = a.attrs['href']
    text = a.string[1:-2] #removing added quotes
    date = a.next_sibling.string
    tweets.append({"name":name
                   ,"title":title
                   ,"link":link
                   ,"body":text
                   ,"date":date})

In [ ]:
with open('data/2017-01-26_Trump_insults_NYT','w') as f:
    f.write(json.dumps(tweets))

In [ ]:
len(tweets)